In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols


file_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/data/final_data/filtered_data_w_ratings.csv'
df = pd.read_csv(file_path)
df['release_date'] = pd.to_datetime(df['release_date'])
df['profit'] = df['worldwide_gross'] - df['production_budget']

# taking the log
min_profit = df['profit'].min()
offset = abs(min_profit) + 1

df['log_profit'] = np.log(df['profit'] + offset)

df['log_production_budget'] = np.log(df['production_budget']+1)


# same period movie metrics
def calculate_same_period_metrics_with_id(index, window=5):  # 5 days before and after
    current_release_date = df.iloc[index]['release_date']
    start_date = current_release_date - pd.DateOffset(days=window)
    end_date = current_release_date + pd.DateOffset(days=window)
    same_period = df[(df['release_date'] >= start_date) & (df['release_date'] <= end_date) & (df.index != index)]
    
    if same_period.empty:
        return pd.Series([0, 0, 0, 0, ''], index=['same_period_indicator', 'same_period_profit', 'same_period_rating', 'same_period_budget', 'same_period_movie_id'])
    
    ## make some modification here (average first then log transformation)
    profit_avg = same_period['profit'].mean()
    log_profit = np.log(offset+profit_avg)
    avg_rating = same_period['averageRating'].mean()
    budget_avg = same_period['production_budget'].mean()
    log_budget = np.log(budget_avg+1)
    same_period_movie_id = ','.join(same_period['tconst'])
    
    return pd.Series([1, log_profit, avg_rating, log_budget, same_period_movie_id], index=['same_period_indicator', 'same_period_profit', 'same_period_rating', 'same_period_budget', 'same_period_movie_id'])


df[['same_period_indicator', 'same_period_profit', 'same_period_rating', 'same_period_budget', 'same_period_movie_id']] = df.index.to_series().apply(calculate_same_period_metrics_with_id)

#### Filter the sample based on Ratings
# filtered_df = df.loc[df['averageRating']>7]

# Regression Model
model_formula = 'log_profit ~ same_period_indicator * same_period_profit + same_period_indicator * same_period_rating + same_period_indicator * same_period_budget'
model = ols(model_formula, data=df).fit()

# Conduct ANOVA analysis
anova_results = sm.stats.anova_lm(model, typ=2)



### 1. All movies vs. all movies

In [2]:
# ANOVA summary & Regression summary
anova_results

,sum_sq,df,F,PR(>F)
same_period_indicator,4.124493e-01,1.0,1.265334e+00,0.260771
same_period_profit,1.037907e-13,1.0,3.184148e-13,0.999999
same_period_indicator:same_period_profit,7.795966e-01,1.0,2.391688e+00,0.122131
same_period_rating,4.893532e+05,1.0,1.501264e+06,0.000000
same_period_indicator:same_period_rating,7.299001e-02,1.0,2.239226e-01,0.636116
same_period_budget,-2.580505e-08,1.0,-7.916610e-08,1.000000
same_period_indicator:same_period_budget,2.138710e-03,1.0,6.561247e-03,0.935448
Residual,6.910370e+02,2120.0,NaN,NaN


In [3]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log_profit   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.247
Date:                Fri, 24 Nov 2023   Prob (F-statistic):              0.289
Time:                        16:27:06   Log-Likelihood:                -1821.7
No. Observations:                2125   AIC:                             3653.
Df Residuals:                    2120   BIC:                             3682.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                   19.0670      0.044    432.867      0.000      18.981      19.153
same_period_indicator                       -1.5537      0.903     -1.721      0.085      -3.324       0.216
same_period_profit                           0.0411      0.027      1.547      0.122      -0.011       0.093
same_period_indicator:same_period_profit     0.0411      0.027      1.547      0.122      -0.011       0.093
same_period_rating                           0.0044      0.009      0.473      0.636      -0.014       0.023
same_period_indicator:same_period_rating     0.0044      0.009      0.473      0.636      -0.014       0.023
same_period_budget                          -0.0007      0.008     -0.081      0.935      -0.017       0.015
same_period_indicator:same_period_budget    -0.0007      0.008     -0.081      0.935      -0.017       0.015
==============================================================================
Omnibus:                     4448.990   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         30556567.858
Skew:                         -17.070   Prob(JB):                         0.00
Kurtosis:                     589.467   Cond. No.                     1.52e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.17e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 2. High ratings vs. high ratings

In [4]:
#### Filter the sample based on Ratings
filtered_df = df.loc[df['averageRating']>7]

# Regression Model
model_formula = 'log_profit ~ same_period_indicator * same_period_profit + same_period_indicator * same_period_rating + same_period_indicator * same_period_budget'
model = ols(model_formula, data=filtered_df).fit()

# Conduct ANOVA analysis
anova_results = sm.stats.anova_lm(model, typ=2)

# ANOVA summary & Regression summary
anova_results

,sum_sq,df,F,PR(>F)
same_period_indicator,5.992000e-01,1.0,2.993973e+00,0.084150
same_period_profit,-1.385235e-13,1.0,-6.921491e-13,1.000000
same_period_indicator:same_period_profit,8.727990e-01,1.0,4.361043e+00,0.037241
same_period_rating,-5.590397e-10,1.0,-2.793307e-09,1.000000
same_period_indicator:same_period_rating,3.413765e-03,1.0,1.705728e-02,0.896138
same_period_budget,-1.401811e-11,1.0,-7.004315e-11,1.000000
same_period_indicator:same_period_budget,2.160737e-03,1.0,1.079638e-02,0.917283
Residual,1.072726e+02,536.0,NaN,NaN


In [5]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log_profit   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2.139
Date:                Fri, 24 Nov 2023   Prob (F-statistic):             0.0748
Time:                        16:27:12   Log-Likelihood:                -329.96
No. Observations:                 541   AIC:                             669.9
Df Residuals:                     536   BIC:                             691.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                   19.3943      0.086    225.265      0.000      19.225      19.563
same_period_indicator                       -3.3453      1.377     -2.430      0.015      -6.050      -0.641
same_period_profit                           0.0840      0.040      2.088      0.037       0.005       0.163
same_period_indicator:same_period_profit     0.0840      0.040      2.088      0.037       0.005       0.163
same_period_rating                           0.0019      0.015      0.131      0.896      -0.027       0.031
same_period_indicator:same_period_rating     0.0019      0.015      0.131      0.896      -0.027       0.031
same_period_budget                          -0.0014      0.013     -0.104      0.917      -0.027       0.025
same_period_indicator:same_period_budget    -0.0014      0.013     -0.104      0.917      -0.027       0.025
==============================================================================
Omnibus:                      109.325   Durbin-Watson:                   1.720
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              176.405
Skew:                           1.287   Prob(JB):                     4.94e-39
Kurtosis:                       4.094   Cond. No.                     1.34e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.98e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Including high rating dummy (All vs. high ratings)

In [6]:
df['high_rating_dummy'] = (df['averageRating'] > 7).astype(int)

model_formula = """
log_profit ~ production_budget + 
         same_period_indicator + 
         same_period_profit + 
         same_period_budget + 
         high_rating_dummy + 
         high_rating_dummy:same_period_indicator + 
         high_rating_dummy:same_period_profit + 
         high_rating_dummy:same_period_budget
"""
model = ols(model_formula, data=df).fit()

anova_results = sm.stats.anova_lm(model, typ=2)
anova_results

,sum_sq,df,F,PR(>F)
production_budget,67.239815,1.0,233.920712,3.531693e-50
same_period_indicator,0.373148,1.0,1.298147,2.546803e-01
same_period_profit,0.262774,1.0,0.914164,3.391206e-01
same_period_budget,0.054948,1.0,0.191159,6.619978e-01
high_rating_dummy,11.717357,1.0,40.763534,2.103388e-10
high_rating_dummy:same_period_indicator,0.180999,1.0,0.629677,4.275624e-01
high_rating_dummy:same_period_profit,0.044436,1.0,0.154587,6.942291e-01
high_rating_dummy:same_period_budget,0.181005,1.0,0.629699,4.275543e-01
Residual,611.112392,2126.0,NaN,NaN


In [7]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log_profit   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     36.23
Date:                Fri, 24 Nov 2023   Prob (F-statistic):           3.51e-54
Time:                        16:27:43   Log-Likelihood:                -1694.1
No. Observations:                2135   AIC:                             3406.
Df Residuals:                    2126   BIC:                             3457.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                  18.7971      0.047    398.750      0.000      18.705      18.890
production_budget                        4.656e-09   3.04e-10     15.294      0.000    4.06e-09    5.25e-09
same_period_indicator                      -0.5639      0.977     -0.577      0.564      -2.481       1.353
same_period_profit                          0.0352      0.056      0.627      0.531      -0.075       0.145
same_period_budget                          0.0001      0.017      0.008      0.993      -0.034       0.034
high_rating_dummy                           0.3640      0.113      3.231      0.001       0.143       0.585
high_rating_dummy:same_period_indicator    -1.5208      1.917     -0.794      0.428      -5.279       2.238
high_rating_dummy:same_period_profit        0.0436      0.111      0.393      0.694      -0.174       0.261
high_rating_dummy:same_period_budget        0.0287      0.036      0.794      0.428      -0.042       0.100
==============================================================================
Omnibus:                     5035.585   Durbin-Watson:                   1.896
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         61707229.640
Skew:                         -22.659   Prob(JB):                         0.00
Kurtosis:                     834.631   Cond. No.                     9.15e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.15e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
df.to_csv('final_data_w_scores.csv')

### Incorporates searching index

In [9]:
import os
import glob

file_path = '/Users/ryan/Documents/GitHub/MGT4187-Project/searching_index/results_supplement'
os.chdir(file_path)
file_ls = glob.glob('results_*')

df = pd.concat(pd.read_csv(file) for file in file_ls)

df['release_date'] = pd.to_datetime(df['release_date'])
df['profit'] = df['worldwide_gross'] - df['production_budget']

# taking the log
min_profit = df['profit'].min()
offset = abs(min_profit) + 1

df['log_profit'] = np.log(df['profit'] + offset)

df['log_production_budget'] = np.log(df['production_budget']+1)

# drop null value
df = df.dropna(subset='search_index')

df = df.reset_index()
# same period movie metrics
def calculate_same_period_metrics_with_id(index, window=5):
    current_release_date = df.iloc[index]['release_date']
    start_date = current_release_date - pd.DateOffset(days=window)
    end_date = current_release_date + pd.DateOffset(days=window)
    same_period = df[(df['release_date'] >= start_date) & (df['release_date'] <= end_date) & (df.index != index)]
    
    if same_period.empty:
        return pd.Series([0, 0, 0, 0, '', 0], index=['same_period_indicator', 'same_period_profit', 'same_period_rating', 'same_period_budget', 'same_period_movie_id', 'same_period_search_index'])
    
    avg_profit = same_period['log_profit'].mean()
    avg_rating = same_period['averageRating'].mean()
    avg_budget = same_period['log_production_budget'].mean()
    same_period_movie_id = ','.join(same_period['tconst'])
    avg_search_index = same_period['search_index'].mean()
    
    return pd.Series([1, avg_profit, avg_rating, avg_budget, same_period_movie_id, avg_search_index], index=['same_period_indicator', 'same_period_profit', 'same_period_rating', 'same_period_budget', 'same_period_movie_id', 'same_period_search_index'])

# Apply the updated function to each movie
df[['same_period_indicator', 'same_period_profit', 'same_period_rating', 'same_period_budget', 'same_period_movie_id', 'avg_search_index']] = df.index.to_series().apply(calculate_same_period_metrics_with_id)

In [25]:
model_formula = 'log_profit ~ same_period_indicator * same_period_profit + same_period_indicator * same_period_rating + same_period_indicator * same_period_budget+ same_period_indicator* avg_search_index'
model = ols(model_formula, data=df).fit()

# Conduct ANOVA analysis
anova_results = sm.stats.anova_lm(model, typ=2)

# ANOVA summary & Regression summary
anova_results

,sum_sq,df,F,PR(>F)
same_period_indicator,1.498520e-01,1.0,9.322003e-01,0.334445
same_period_profit,-2.102289e-08,1.0,-1.307793e-07,1.000000
same_period_indicator:same_period_profit,3.436130e-01,1.0,2.137549e+00,0.143937
same_period_rating,-6.091282e-11,1.0,-3.789268e-10,1.000000
same_period_indicator:same_period_rating,5.901860e-02,1.0,3.671433e-01,0.544656
same_period_budget,1.144344e-08,1.0,7.118742e-08,0.999787
same_period_indicator:same_period_budget,1.895792e-01,1.0,1.179335e+00,0.277663
avg_search_index,2.919726e-04,1.0,1.816305e-03,0.966012
same_period_indicator:avg_search_index,2.253714e-03,1.0,1.401992e-02,0.905762
Residual,2.454666e+02,1527.0,NaN,NaN


In [26]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log_profit   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.726
Date:                Fri, 24 Nov 2023   Prob (F-statistic):              0.125
Time:                        16:08:02   Log-Likelihood:                -771.14
No. Observations:                1533   AIC:                             1554.
Df Residuals:                    1527   BIC:                             1586.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                   19.1637      0.030    643.046      0.000      19.105      19.222
same_period_indicator                       -1.4963      0.710     -2.107      0.035      -2.889      -0.103
same_period_profit                           0.0306      0.021      1.462      0.144      -0.010       0.072
same_period_indicator:same_period_profit     0.0306      0.021      1.462      0.144      -0.010       0.072
same_period_rating                           0.0046      0.008      0.606      0.545      -0.010       0.020
same_period_indicator:same_period_rating     0.0046      0.008      0.606      0.545      -0.010       0.020
same_period_budget                           0.0071      0.007      1.086      0.278      -0.006       0.020
same_period_indicator:same_period_budget     0.0071      0.007      1.086      0.278      -0.006       0.020
avg_search_index                          -4.47e-05      0.000     -0.118      0.906      -0.001       0.001
same_period_indicator:avg_search_index    -4.47e-05      0.000     -0.118      0.906      -0.001       0.001
==============================================================================
Omnibus:                      422.202   Durbin-Watson:                   1.768
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1002.022
Skew:                           1.504   Prob(JB):                    2.59e-218
Kurtosis:                       5.576   Cond. No.                     1.85e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.85e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
df.to_csv('final_data_w_scores_and_searching_index.csv')